In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
data = [i.strip('\n').split('\t') for i in open('/content/drive/My Drive/Colab Notebooks/openclass_prize.train', 'r', encoding='utf8', errors='ignore')]
print(data[0])
print(" ")
print(data[1])

['What is the Turing Test?', 'The original Turing test involved three entities communicating with each other through computer terminals in separate rooms - a computer program trying to pass itself off as a human, a human trying to prove that she or he is human, and a human judge deciding which of the competitors is human and which a machine. This is different from the popular idea that it\'s about a human judging whether one subject is human or machine. The reason it\'s different is that Turing\'s original proposal was not intended as a practical test, but rather as a thought experiment making a philosophical argument - that it is impossible to determine whether something is "intelligent" better than by judging its linguistic interactions, and that therefore a machine that sounds intelligent has to be considered truly intelligent. And by "intelligent," Turing meant, like a human, with consciousness.', 'Which of the following is NOT an objection to the Turing Test mentioned in the readi

In [3]:
X = []
Y = []
for i in range(len(data)):
    if i % 2 == 0:
        X.append(' '.join(data[i]))
    else:
        Y.append(data[i])

print(X[0:3])
print(" ")
print(Y[0:5])

['What is the Turing Test? The original Turing test involved three entities communicating with each other through computer terminals in separate rooms - a computer program trying to pass itself off as a human, a human trying to prove that she or he is human, and a human judge deciding which of the competitors is human and which a machine. This is different from the popular idea that it\'s about a human judging whether one subject is human or machine. The reason it\'s different is that Turing\'s original proposal was not intended as a practical test, but rather as a thought experiment making a philosophical argument - that it is impossible to determine whether something is "intelligent" better than by judging its linguistic interactions, and that therefore a machine that sounds intelligent has to be considered truly intelligent. And by "intelligent," Turing meant, like a human, with consciousness. Which of the following is NOT an objection to the Turing Test mentioned in the reading? In

In [0]:
#may not need any of this with the glove embedding I am using
from nltk.stem.snowball import SnowballStemmer
import re

def clean_data(dataset):
    clean_data = []
    sno = SnowballStemmer('english')

    for entry in dataset:
        lowered = entry.lower()
        sans_special_chars = re.sub(r'\W',' ', lowered)
        sans_extra_spaces = re.sub(r'\s+',' ', sans_special_chars)
        sans_underscore = sans_extra_spaces.replace('_', ' ')
        split = sans_underscore.split()
        stemmed = ' '.join([sno.stem(word) for word in split])
        clean_data.append(stemmed)

    return clean_data

In [0]:
# clean_X = clean_data(X)
# print(clean_X[0:3])

In [6]:
labels = [item for sublist in Y for item in sublist]
print(labels[0:5])

['Turing test', 'Alan Turing', 'Computational linguistics', 'Language philosophy', 'Artificial Intelligence']


In [7]:
print(len(labels))

716


In [0]:
import numpy as np

sample_labels = np.zeros((len(Y), len(labels)))

for i in range(len(Y)):
    for j in range(len(Y[i])):
        for k in range(len(labels)):
            if Y[i][j] == labels[k]:
                sample_labels[i][k] = 1
                break

# sample_labels[0]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, sample_labels, test_size=0.2)
print("Length of X: ", len(X))
print("Length of Y: ", len(Y))
print("Length of X_train: ", len(X_train))
print("Length of Y_train: ", len(Y_train))

Length of X:  163
Length of Y:  163
Length of X_train:  130
Length of Y_train:  130


In [10]:
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.sequence as kps
NUM_WORDS = 120000

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_train)

train_indices = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
train_indices = kps.pad_sequences(train_indices, padding='post')

test_indices = tokenizer.texts_to_sequences(X_test)
test_indices = kps.pad_sequences(test_indices, padding='post', maxlen=len(train_indices[0]))

print("Train Data: ", train_indices[0:3])
print("Test Data: ", test_indices[0:3])

Using TensorFlow backend.


Train Data:  [[ 52   7   1 ...   0   0   0]
 [ 80  11 569 ...   0   0   0]
 [ 52   7   1 ...   0   0   0]]
Test Data:  [[1062  881  889 ...    0    0    0]
 [  52   11 1549 ...    0    0    0]
 [  52    7    1 ...    0    0    0]]


In [0]:
# import tensorflow as tf
from tensorflow.keras import models, layers
EMBEDDING_DIM = 100
embeddings_index = {}

f = open("/content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt", 'r', encoding='utf8', errors='ignore')
for line in f:
    values = line.split()
    word = ''.join(values[:-EMBEDDING_DIM])
    coefs = np.asarray(values[-EMBEDDING_DIM:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = layers.Embedding(len(word_index) + 1,
                        EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=len(train_indices[0]),
                        trainable=True,
                        mask_zero=True)

In [0]:
from tensorflow.keras import callbacks
def create_model():
    model = models.Sequential()
    model.add(embedding_layer)
    model.add(layers.Bidirectional(layers.GRU(500)))
    model.add(layers.Dense(325, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(len(labels), activation ='sigmoid'))

    model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return [model, {'batch_size' : 16, 'callbacks' :[callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)]}] # callback only used here to restore best weights if need be

In [14]:
model, kwargs = create_model()

kwargs.update(x=train_indices, y=Y_train, epochs=2, validation_data=(test_indices, Y_test))
model.fit(**kwargs)

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("modle.h5")

Epoch 1/2
9/9 [==============================] - 887s 99s/step - loss: 0.3488 - accuracy: 0.0000e+00 - val_loss: 0.0467 - val_accuracy: 0.0000e+00
Epoch 2/2
9/9 [==============================] - 952s 106s/step - loss: 0.0495 - accuracy: 0.0385 - val_loss: 0.0444 - val_accuracy: 0.0000e+00


In [23]:
predictions_probs = model.predict(test_indices)
predictions_probs[0]

array([1.02001429e-03, 4.30285931e-04, 1.89357996e-03, 4.92006540e-04,
       8.14795494e-04, 1.37981772e-03, 2.12305784e-03, 1.98203325e-03,
       2.25666165e-03, 8.69604945e-03, 7.90327787e-04, 1.12658739e-03,
       8.46087933e-04, 8.63584876e-03, 1.50719285e-03, 1.42386943e-01,
       1.72395408e-02, 4.36037779e-03, 2.17379630e-02, 4.36782837e-03,
       1.98751688e-04, 1.02308393e-03, 3.84458899e-03, 8.41557980e-04,
       2.42799520e-04, 4.16913629e-03, 4.46081161e-04, 1.67876482e-04,
       1.23298756e-04, 2.25025415e-03, 1.26916170e-03, 4.23938036e-04,
       1.48150325e-03, 2.14469433e-03, 2.72601843e-04, 3.38862240e-02,
       6.61563873e-03, 9.60245728e-03, 6.42584264e-02, 1.02269650e-02,
       1.65979981e-01, 3.00467014e-04, 1.42228603e-03, 1.45578384e-03,
       2.34544277e-04, 4.81185317e-03, 3.47891450e-03, 1.59570575e-03,
       6.26891851e-04, 5.55565953e-03, 2.21043825e-04, 2.24055648e-02,
       1.20645761e-03, 2.99870968e-04, 6.73031807e-03, 4.08500433e-04,
      

In [26]:
predictions = np.where(predictions_probs > 0.01, 1, 0)
predictions[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, _, __ = precision_recall_fscore_support(Y_test, predictions, average='macro')
print((5 *  precision * recall) / (4 * (precision+recall)))
precision, recall, _, __ = precision_recall_fscore_support(Y_test, predictions, average='micro')
print((5 *  precision * recall) / (4 * (precision+recall)))
precision, recall, _, __ = precision_recall_fscore_support(Y_test, predictions, average='weighted')
print((5 *  precision * recall) / (4 * (precision+recall)))

0.001437053255592691
0.050699300699300696
0.02435109904569315


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
